In [ ]:
"""
Koden henter laksedata fra SSB og gjør dataen om til en tabell for alle verdiene som brukes videre i prosjektet ved hjelp av Pandas.
"""

import requests
import pandas as pd
from itertools import product

# URL til SSB API for tabell 03024
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    """Henter data fra SSB API og returnerer JSON-responsen."""
    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    response = requests.post(api_url, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Feil ved henting av data: {response.status_code}\n{response.text}")

def process_data(data):
    """Prosesserer JSON-data og returnerer en formatert Pandas DataFrame."""
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    
    # Oppretter en liste av kombinasjoner av dimensjoner
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))
    
    # Lager en DataFrame med riktige dimensjoner
    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values  # Knytter verdiene til riktige rader
    
    # Pivoterer data for å få ønsket struktur
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()
    
    # Gir kolonnene mer forståelige navn med riktig rekkefølge
    df_pivot.columns = ["År og ukenr.", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)", "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]
    
    return df_pivot

In [8]:
def main():
    """Hovedfunksjon for å hente og prosessere data."""
    try:
        data = fetch_data()
        df_pivot = process_data(data)
        print(df_pivot.to_string(index=False))
        print("Data hentet, sortert og formatert for visning.")
    except Exception as e:
        print(e)

if __name__ == "__main__":
    main()

År og ukenr.  Fersk laks - Kilospris  Fersk laks - Vekt (tonn)  Frosset laks - Kilospris  Frosset laks - Vekt (tonn)
     2000U01                   30.98                    3728.0                     32.54                       383.0
     2000U02                   31.12                    4054.0                     33.63                       216.0
     2000U03                   31.03                    4043.0                     36.06                       633.0
     2000U04                   30.95                    3730.0                     34.27                       393.0
     2000U05                   31.30                    3831.0                     33.91                       453.0
     2000U06                   32.53                    4415.0                     35.30                       265.0
     2000U07                   32.46                    4617.0                     34.49                       344.0
     2000U08                   32.19                    4463.0  